In [1]:
import PyPDF2
import re
import os
import gradio as gr

# --- Configuración ---
PDF_PATH = "Prueba3.pdf"
MIN_MATCH_SCORE = 2


# --- Lógica del Chatbot (Funciones sin cambios) ---

def extract_text_from_pdf(pdf_path):
    """Extrae texto de cada página de un archivo PDF."""
    pages_text = []
    if not os.path.exists(pdf_path):
        print(f"Error: El archivo PDF '{pdf_path}' no fue encontrado.")
        raise FileNotFoundError(f"Error crítico: No se encontró el archivo '{pdf_path}'. Asegúrate de que está en el directorio correcto.")
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            print(f"Leyendo {num_pages} páginas del PDF...")
            for page_num in range(num_pages):
                try:
                    page = reader.pages[page_num]
                    text = page.extract_text()
                    pages_text.append(text if text else "")
                except Exception as page_e:
                    print(f"Advertencia: No se pudo extraer texto de la página {page_num + 1}. Error: {page_e}")
                    pages_text.append("")
            print("Extracción de texto completada.")
    except Exception as e:
        print(f"Error al leer el PDF: {e}")
        raise RuntimeError(f"Error al procesar el PDF: {e}")
    return pages_text

def preprocess_text(text):
    """Limpia y normaliza el texto."""
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\sáéíóúüñ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def find_relevant_page(query, pages_text):
    """Encuentra la página más relevante basado en palabras clave."""
    processed_query = preprocess_text(query)
    query_words = set(processed_query.split())

    if not query_words:
        return "Por favor, introduce una consulta válida."

    best_page_index = -1
    best_score = 0

    for i, page_content in enumerate(pages_text):
        processed_page = preprocess_text(page_content)
        if not processed_page:
             continue
        page_words = set(processed_page.split())
        current_score = len(query_words.intersection(page_words))

        important_keywords = {"vacaciones", "licencia", "enfermedad", "salario", "despido", "ética", "confidencialidad", "horario", "jurado", "testigo", "seguridad"}
        bonus_score = sum(1 for keyword in important_keywords if keyword in query_words and keyword in page_words)
        current_score += bonus_score * 2

        if current_score > best_score:
            best_score = current_score
            best_page_index = i

    if best_score >= MIN_MATCH_SCORE and best_page_index != -1:
        page_number = best_page_index + 1
        response = (
            f"**Información encontrada (Página {page_number} del Manual):**\n\n"
            # Usamos Markdown para potencialmente mejorar formato (aunque Gradio maneja bien texto plano)
            f"```\n{pages_text[best_page_index].strip()[:800]}...\n```"
            f"\n\n*(Nota: Este es un extracto de la página completa ({best_score} puntos). Puede contener más detalles.)*"
        )
        return response
    else:
        return "Lo siento, no pude encontrar información específica sobre tu consulta en el Manual del Empleado. Intenta reformular tu pregunta con otras palabras clave o consulta directamente con Recursos Humanos."

# --- Carga del Manual (Hacerlo una sola vez al inicio) ---
try:
    print("Iniciando carga del Manual del Empleado...")
    manual_pages_content = extract_text_from_pdf(PDF_PATH)
    if not manual_pages_content:
         print("Advertencia: El manual parece estar vacío o no se pudo leer ninguna página.")
         manual_pages_content = []
except Exception as startup_e:
     print(f"Error fatal durante la carga inicial: {startup_e}")
     manual_pages_content = []

# --- Lógica de Interacción para Gradio Blocks ---
def user(user_message, history):
    """Añade el mensaje del usuario al historial."""
    # Si el historial es None (primera vez), inicialízalo
    if history is None:
        history = []
    return "", history + [[user_message, None]] # Retorna textbox vacío y historial actualizado

def bot(history):
    """Genera la respuesta del bot basada en el último mensaje del usuario."""
    if not manual_pages_content:
         bot_message = "Error: No se pudo cargar la información del Manual del Empleado. Por favor, contacta al administrador."
    else:
        user_message = history[-1][0] # Obtiene el último mensaje del usuario
        print(f"Procesando consulta: {user_message}")
        bot_message = find_relevant_page(user_message, manual_pages_content)
        print(f"Respuesta generada: {bot_message[:100]}...")

    history[-1][1] = bot_message # Actualiza el placeholder del bot en el historial
    return history

# --- Creación de la Interfaz Gradio con Blocks ---
print("Creando la interfaz Gradio con Blocks...")

with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="sky"), title="Chatbot RRHH") as demo:
    gr.Markdown(
        """
        # 💬 Chatbot RRHH - Asociación Pro Desarrollo Comunal del Patio, Inc.
        Consulta información del Manual del Empleado. Escribe tu pregunta abajo.
        """
    )
    # El componente Chatbot ocupará el ancho disponible en esta columna principal
    chatbot = gr.Chatbot(
        [], # Historial inicial vacío
        elem_id="chatbot",
        label="Conversación",
        bubble_full_width=False, # Burbujas no ocupan todo el ancho
        height=400 # Altura aumentada para compensar el ancho
        )

    # Fila para la entrada de texto y el botón
    with gr.Row():
        # El Textbox ocupa la mayor parte de la fila (scale=8)
        txt = gr.Textbox(
            scale=8,
            show_label=False,
            placeholder="Escribe tu pregunta aquí (ej: '¿Cuántos días de vacaciones tengo?', 'Política sobre enfermedad') y presiona Enter",
            container=False,
        )
        # El botón ocupa una parte menor (scale=1)
        submit_btn = gr.Button("Enviar", variant="primary", scale=1)

    # Botón para limpiar (opcional pero útil)
    clear_btn = gr.ClearButton([txt, chatbot], value="Limpiar Chat")

    # Ejemplos
    gr.Examples(
        examples=[
            "¿Cuál es la política de vacaciones?",
            "Información sobre licencia por enfermedad",
            "¿Qué dice el código de ética?",
            "Hostigamiento sexual",
            "¿Hay bono de navidad?",
            "Uso del teléfono personal",
            "Periodo probatorio"
        ],
        inputs=txt, # Los ejemplos se cargarán en el Textbox
        label="Preguntas de Ejemplo"
    )

    # --- Conexión de Eventos ---
    # 1. Cuando el usuario envía un mensaje (Enter en Textbox)
    txt.submit(user, [txt, chatbot], [txt, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    # 2. Cuando el usuario hace clic en el botón Enviar
    submit_btn.click(user, [txt, chatbot], [txt, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

# --- Lanzar la aplicación ---
if __name__ == "__main__":
    print("Lanzando la aplicación Gradio... Accede desde tu navegador a la URL local.")
    demo.queue() # Habilitar cola para manejar múltiples solicitudes si es necesario
    demo.launch(share=False, inbrowser=True, pwa=True)

Iniciando carga del Manual del Empleado...
Leyendo 29 páginas del PDF...
Extracción de texto completada.
Creando la interfaz Gradio con Blocks...


C:\Users\myazi\AppData\Local\Temp\ipykernel_1856\1027657304.py:128: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
C:\Users\myazi\AppData\Local\Temp\ipykernel_1856\1027657304.py:128: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


Lanzando la aplicación Gradio... Accede desde tu navegador a la URL local.
* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Procesando consulta: vacaciones
Respuesta generada: **Información encontrada (Página 3 del Manual):**

```
TABLA DE CONTENIDO
MISIÓN Y VISIÓN..............
